In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score

In [ ]:
SEED = 42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('./drive/My Drive/data/train.csv')
test_df = pd.read_csv('./drive/My Drive/data/test.csv')

In [ ]:
train_df.head(15)

,sentence,language
0,"Pensez à la communication , le discours , les ...",fr
1,"Můžete si ji pronajmout , vzít na splátky , ko...",cs
2,"Každý starosta pochopil , že když mají tyto fo...",cs
3,"Det är ytterligare bevis , men ändå — Jag krit...",sv
4,كان الأمر لا يصدق .,ar
5,"Na primjer , pjesnik ga opisuje ovako .",hr
6,Semua rerumputan itu sekarang menutupi tanah s...,id
7,Det är en enorm utmaning för oss att göra det .,sv
8,Ono što ćete vidjeti - trebat ćete skočiti ovd...,hr
9,( Alqışlar ) Excel-dən istifadə etmişəm .,az


In [ ]:
lang_count = (
    train_df.language.value_counts()
    .to_frame()
    .reset_index()
)

lang_count.head()

,index,language
0,ru,181970
1,en,157655
2,fr,148585
3,vi,113193
4,ar,110032


In [ ]:
train_df[train_df.language == "ru"]

,sentence,language
10,После увольнения я решил собрать своих друзей ...,ru
13,"Ты знаешь , что это не гнев заставил моё сердц...",ru
26,гэта што калі мы не яны як але ён каб дзякуй б...,ru
40,"Ради всех имён , которые я никогда не узнаю , ...",ru
91,що це тому ми чи дуже які щоб дякую якщо було ...,ru
...,...,...
3159578,гэта што калі мы не яны як але ён каб дзякуй б...,ru
3159582,що це тому ми чи дуже які щоб дякую якщо було ...,ru
3159591,Некоторые действительно на грани вымирания .,ru
3159619,що це тому ми чи дуже які щоб дякую якщо було ...,ru


In [ ]:
from collections import Counter
langCounter = {}
size = 2046*59
x = [] #x = [[None] * 2 for i in range(size)]
i = 0
j = 0
while 1 > 0:
  if train_df.language[i] not in langCounter.keys():
    langCounter[train_df.language[j]] = 0
  if langCounter[train_df.language[j]] < 2046:
    x.append([train_df.sentence[j],train_df.language[j]])
    langCounter[train_df.language[j]] += 1
    i+= 1
  j += 1
  #print(i)
  if i == size:
    break

In [ ]:
x[len(x)-1][0]

'Мінеки , тағы бірнешеуі .'

In [ ]:
delcount = 0
for i in range(len(x)-1):
  if 'шо' in x[i-delcount][0] or 'це' in x[i-delcount][0] or 'що' in x[i-delcount][0] or 'ми' in x[i-delcount][0] or 'чи' in x[i-delcount][0]:
    del x[i-delcount]
    delcount+=1
len(x)

113355

In [ ]:
x=pd.DataFrame(x)
x.rename(columns={0: 'sentence', 1: 'language'}, inplace=True)


In [ ]:
train_df.language

0             fr
1             cs
2             cs
3             sv
4             ar
           ...  
3159628       ja
3159629       bg
3159630       it
3159631    pt-br
3159632    fr-ca
Name: language, Length: 3159633, dtype: object

In [ ]:
train_df[(train_df.language == 'ru')]
if 'які ' in train_df.sentence[3159619]:
  print("sad(")

sad(


In [ ]:
train_df.sentence[3159619]

'що це тому ми чи дуже які щоб дякую якщо було вона від мене ось'

In [ ]:
lang_count[lang_count['index'].apply(lambda x: x in {"ru", "kk"})]

,index,language
0,ru,181970
58,kk,2046


In [ ]:
lang_count.shape

(59, 2)

# Model

In [ ]:
xx = x.values[:,  0]
label_encoder = LabelEncoder().fit(x.values[:, 1])
y = label_encoder.transform(x.values[:, 1])

x_train, x_test, y_train, y_test = train_test_split(xx, y, test_size=0.2, random_state=SEED)

In [ ]:
pipe = Pipeline([
  ('vectorizer', TfidfVectorizer(analyzer='char_wb',ngram_range=(2,3))), #analyzer=str
    ('model', SGDClassifier(loss='modified_huber',l1_ratio=0.1))])#SGDClassifier(loss='modified_huber',l1_ratio=0.1) 

In [ ]:
x.values[: 1]

array([['Pensez à la communication , le discours , les gestes , écrire , le langage des signes , ils sont tous faits par l&apos; intermédiaire de contractions musculaires .',
        'fr']], dtype=object)

In [ ]:
for i in range(x_train.shape[0]):
    x_train[i] =  x_train[i].lower()
for i in range(x_test.shape[0]):
    x_test[i] = x_test[i].lower()

In [ ]:
pipe.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char_wb', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(2, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_p...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.1,
                               learning_rate='optimal', loss='modified_huber',
   

In [ ]:
predictions = pipe.predict(x_train)

In [ ]:
balanced_accuracy_score(y_train, predictions)

0.9644165111367429

In [ ]:
test_predictions = pipe.predict(x_test)

In [ ]:
balanced_accuracy_score(y_test, test_predictions)

0.9333722680495792

In [ ]:
submit_predictions = pipe.predict(test_df.sentence.values)

KeyboardInterrupt: ignored

In [ ]:
test_df['language'] = label_encoder.classes_[submit_predictions]

test_df.head()

In [ ]:
(
    test_df[['index',  'language']]
    .to_csv('/content/sample_data/sample_submission.csv', index=False)
)